In [1]:
import sympy as sp
import numpy as np

In [53]:
# basis vectors in x, y, and t
x=sp.Symbol('x')
y=sp.Symbol('y')
t=sp.Symbol('t')
#model and data amplitudes
Ad=sp.Symbol('A_d')
Am=sp.Symbol('A_m')

# domain width
L=sp.Symbol('L', positive=True, finite=True)
# domain time dinension
T=sp.Symbol('T', positive=True, finite=True)

# data errors:
s=sp.Symbol('\sigma_d')
# data density:
rd=sp.Symbol('rho', positive=True, finite=True)
# wavenumbers in the x, y, and t dimensions
kx=sp.Symbol('k_x', integer=True, positive=True)
ky=sp.Symbol('k_y', integer=True, positive=True)
kt=sp.Symbol('k_t', integer=True, positive=True)
# Weighting on d3z/dx2dt
Wxxt=sp.Symbol('W_{xxt}')
# Weighting on d2x/dt2
Wtt=sp.Symbol('W_{tt}')


In [54]:
tau=sp.Symbol('tau')
lam=sp.Symbol('\lambda')
tau*lam

\lambda*tau

In [55]:
# basis functions:
bf =sp.sin(2*kx*sp.pi*x/L) * sp.sin(2*ky*sp.pi*y/L) * sp.sin(2*kt*sp.pi*t/T) 
# model is the basis function times Am
m=Am * bf
# data is basis function times Ad
d=Ad * bf

In [56]:
# derivatives of the basis functions
D2xtm = sp.diff(sp.diff(sp.diff(m, x),x),t)
D2ytm = sp.diff(sp.diff(sp.diff(m, y),y),t)
Dxytm = sp.diff(sp.diff(sp.diff(m, x),y),t)
D2tm = sp.diff(sp.diff(m, t),t)

In [57]:
# Combined residual:
R=sp.integrate(\
               sp.integrate(\
    sp.integrate( rd*((d-m)/s)**2 + (Wxxt*D2xtm)**2 + (Wxxt*D2ytm)**2 + 2*(Wxxt*Dxytm)**2+(Wtt*D2tm)**2, (x, 0, L)),\
    (y, 0, L)), 
               (t, 0, T))

All of the terms inside the integral have units of $m^{-2} yr^{-1}$, so that R is unitless, so every quantity inside parenthesis has units of $m yr^{-1/2}$. 
- $A_d$ and $A_m$ have units of $m$.
- $\rho_d$ has units of $m^{-2} yr^{-1}$ (points per meter squared per year), so $\rho_d(d-m)/\sigma$ has units of $m^{-2} yr^{-1}$.
- $\partial^3 z / \partial x^2 \partial t$ has units of $m^{-1} yr^{-1}$  
- $W_{xxt}$ has units of $yr^{1/2}$ 
- $\partial^2 z /\partial t^2$ has units of $m yr^{-2}$
- $W_{tt}$ has units of $m^{-2} yr^{-3/2}$ 
- $k_x$, $k_y$, and $k_t$ are all unitless


In [76]:
R

2*pi**4*A_m**2*L**2*W_{tt}**2*k_t**4/T**3 + 8*pi**6*A_m**2*W_{xxt}**2*k_t**2*k_x**4/(L**2*T) + 16*pi**6*A_m**2*W_{xxt}**2*k_t**2*k_x**2*k_y**2/(L**2*T) + 8*pi**6*A_m**2*W_{xxt}**2*k_t**2*k_y**4/(L**2*T) + rho*(A_d**2*L**2*T/8 - A_d*A_m*L**2*T/4 + A_m**2*L**2*T/8)/\sigma_d**2

In [58]:
# solve for model amplitude that minimizes the combined residual:
A_best=sp.solve(sp.diff(R, Am), Am)[0]
A_best

A_d*L**4*T**4*rho/(L**4*T**4*rho + 16*pi**4*L**4*W_{tt}**2*\sigma_d**2*k_t**4 + 64*pi**6*T**2*W_{xxt}**2*\sigma_d**2*k_t**2*k_x**4 + 128*pi**6*T**2*W_{xxt}**2*\sigma_d**2*k_t**2*k_x**2*k_y**2 + 64*pi**6*T**2*W_{xxt}**2*\sigma_d**2*k_t**2*k_y**4)

In [59]:
# simplify the denominator:
sp.expand(1/(A_best/(Ad)))

1 + 16*pi**4*W_{tt}**2*\sigma_d**2*k_t**4/(T**4*rho) + 64*pi**6*W_{xxt}**2*\sigma_d**2*k_t**2*k_x**4/(L**4*T**2*rho) + 128*pi**6*W_{xxt}**2*\sigma_d**2*k_t**2*k_x**2*k_y**2/(L**4*T**2*rho) + 64*pi**6*W_{xxt}**2*\sigma_d**2*k_t**2*k_y**4/(L**4*T**2*rho)

It's not clear how best to simplify this expression, but working by hand gives this expression as equivalent to A_best

In [49]:
sp.simplify(Ad*rd/s**2 /  (rd/s**2 + 16*sp.pi**4*(Wtt**2*kt**4/T**4 + 4*sp.pi**2*kt**2*Wxxt**2*(kx**2+ky**2)**2/(L**4*T**2))))

A_d*L**4*T**4*rho/(L**4*T**4*rho + 16*pi**4*\sigma_d**2*k_t**2*(L**4*W_{tt}**2*k_t**2 + 4*pi**2*T**2*W_{xxt}**2*(k_x**2 + k_y**2)**2))

...or better yet...

In [68]:
Ad/(1+16*sp.pi**4*s**2*kt**2/rd*(Wtt**2*kt**2/T**4 + 4*sp.pi**2*T**2*Wxxt**2*(kx**2+ky**2)**2/(L**4*T**4)))

A_d/(16*pi**4*\sigma_d**2*k_t**2*(W_{tt}**2*k_t**2/T**4 + 4*pi**2*W_{xxt}**2*(k_x**2 + k_y**2)**2/(L**4*T**2))/rho + 1)

Writing this in terms of wavelength ($ L (k_x^2+ky^2)^{-2}$) and period ($\tau = T/k_t$), 

In [69]:
Ad/(1+16*sp.pi**4*s**2/rd*(Wtt**2/tau**4 + 4*sp.pi**2*Wxxt**2/(lam**4*tau**2)))

A_d/(16*pi**4*\sigma_d**2*(W_{tt}**2/tau**4 + 4*pi**2*W_{xxt}**2/(\lambda**4*tau**2))/rho + 1)

To choose a value of $W_{tt}$, we look for the wavelength where $A_{best} = A_d/2$, or
$$\frac{16 \pi^4 \sigma_d^2}{\rho_d} \left(\frac{W_{tt}^2}{\tau^4} + \frac{4\pi^2W_{xxt}^2}{\lambda^4\tau^2}\right) =1\,$$


For example, to suppress oscillations of period  < 0.125 yr, for data errors of 0.1 m, with 1 data point /3 km, with the temporal curvature alone, we calculate:

$$ \frac{W_{tt}^2}{\tau^4} = \frac{\rho}{16 \pi^4 \sigma_d^2}$$

$$ W_{tt} = \frac{\tau^2\rho^{1/2}}{4 \pi^2 \sigma_d}$$

In [75]:
taui=0.125  # 1/8 year period
Ti=1 # 1 year
kti = Ti/taui
rho_tr=4/3000   # 4 measurements/year/3 km
si=0.1 # 0.1 m error
Wtti = np.sqrt((Ti**4*rho_tr)/(16*np.pi**4*kti**4*si**2))
print(1/Wtti)
print(1/(taui**2*rho_tr**0.5/(4*np.pi**2*si)))

6919.4303540850005
6919.4303540850005


We only have one independent bias value every 10 km at Jakobshavn latitude, and per-track errors might be as large as 10 m geolocation error * slope of 0.05 -> sigma of 0.5

In [12]:
tau=0.125  # 1/8 year period
Ti=1 # 1 year
kti = Ti/tau
rho_tr=4/10000   # 4 measurements/year/3 km
si=5 # 0.5 m error
Wtti = np.sqrt((Ti**4*rho_tr)/(16*np.pi**4*kti**4*si**2))
print(1/Wtti)

631654.6816697188


In [225]:
# if we want to suppress 1-km scale variations in dh/dt at a temporal scale of 0.25 yr:
Li=1000
wvl=500
Ti=1
tau=0.25
kti=Ti/tau
kxi=Li/wvl
Wxxi = Li**2*Ti/(8*np.pi**3*si*kti*kxi**2)
print(1/Wxxi)

0.0003968803415078377
